In [1]:
import random
import math
from enum import Enum
import networkx as nx

from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
from mesa.space import NetworkGrid

In [2]:
class State(Enum):
    SUSCEPTIBLE = 0
    BURNT = 1
    UNBURNT = 2
    
def number_state(model, state):
    return sum([1 for a in model.grid.get_all_cell_contents() if a.state is state])

def number_burnt(model):
    return number_state(model, State.BURNT)

def number_susceptible(model):
    return number_state(model, State.SUSCEPTIBLE)

def number_unburnt(model):
    return number_state(model, State.UNBURNT)

class NetworkOnFire(Model):
    """A Forest Fire model with some number of agents"""

    def __init__(self, num_nodes=10, avg_node_degree=3, initial_outbreak_size=1, fire_spread_chance=0.4,
                fire_check_frequency=0.4, recovery_chance=0.3, fire_resistance_chance=0.5):

        self.num_nodes = num_nodes
        prob = avg_node_degree / self.num_nodes
        self.G = nx.erdos_renyi_graph(n=self.num_nodes, p=prob)
        self.grid = NetworkGrid(self.G)
        self.schedule = RandomActivation(self)
        self.initial_outbreak_size = initial_outbreak_size if initial_outbreak_size <= num_nodes else num_nodes
        self.fire_spread_chance = fire_spread_chance
        self.fire_check_frequency = fire_check_frequency
        self.recovery_chance = recovery_chance
        self.fire_resistance_chance = fire_resistance_chance
        
        self.datacollector = DataCollector({"Burnt": number_burnt,
                                            "Susceptible": number_susceptible,
                                            "Unburnt": number_unburnt})

        # Create agents
        for i, node in enumerate(self.G.nodes()):
            a = FireAgent(i, self, State.SUSCEPTIBLE, self.fire_spread_chance, self.fire_check_frequency,
                           self.recovery_chance, self.fire_resistance_chance)
            self.schedule.add(a)
            # Add the agent to the node
            self.grid.place_agent(a, node)
            
            
        # Burn some nodes
        burnt_nodes = random.sample(self.G.nodes(), self.initial_outbreak_size)
        for a in self.grid.get_cell_list_contents(burnt_nodes):
            a.state = State.BURNT

        self.running = True
        self.datacollector.collect(self)
        
        
    def unburnt_susceptible_ratio(self):
        try:
            return number_state(self, State.UNBURNT) / number_state(self, State.SUSCEPTIBLE)
        except ZeroDivisionError:
            return math.inf

    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)

    def run_model(self, n):
        for i in range(n):
            self.step()

class FireAgent(Agent):
    def __init__(self, unique_id, model, initial_state, fire_spread_chance, fire_check_frequency,
                 recovery_chance, fire_resistance_chance):
        super().__init__(unique_id, model)

        self.state = initial_state

        self.fire_spread_chance = fire_spread_chance
        self.fire_check_frequency = fire_check_frequency
        self.recovery_chance = recovery_chance
        self.fire_resistance_chance = fire_resistance_chance

    def try_to_burn_neighbors(self):
        neighbors_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
        susceptible_neighbors = [agent for agent in self.model.grid.get_cell_list_contents(neighbors_nodes) if
                                 agent.state is State.SUSCEPTIBLE]
        for a in susceptible_neighbors:
            if random.random() < self.fire_spread_chance:
                a.state = State.BURNT

    def check_resistance(self):
        if random.random() < self.gain_resistance_chance:
            self.state = State.UNBURNT
            
    def try_remove_burn(self):
        # Try to remove
        if random.random() < self.recovery_chance:
            # Success
            self.state = State.SUSCEPTIBLE
            self.check_resistance()
        else:
            # Failed
            self.state = State.BURNT

    def try_check_situation(self):
        if random.random() < self.fire_check_frequency:
            # Checking...
            if self.state is State.BURNT:
                self.try_remove_burn()

    def step(self):
        if self.state is State.BURNT:
            self.try_to_burn_neighbors()
        self.try_check_situation()

In [3]:
import math

from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter
from mesa.visualization.modules import ChartModule
from mesa.visualization.modules import NetworkModule
from mesa.visualization.modules import TextElement
#from .model import NetworkOnFire, State, number_burnt

In [4]:
def network_portrayal(G):
    # The model ensures there is always 1 agent per node

    def node_color(agent):
        return {
            State.BURNT: '#FF0000',
            State.SUSCEPTIBLE: '#008000'
        }.get(agent.state, '#808080')

    def edge_color(agent1, agent2):
        if State.UNBURNT in (agent1.state, agent2.state):
            return '#000000'
        return '#e8e8e8'

    def edge_width(agent1, agent2):
        if State.UNBURNT in (agent1.state, agent2.state):
            return 3
        return 2
    
    def get_agents(source, target):
        return G.node[source]['agent'][0], G.node[target]['agent'][0]

    portrayal = dict()
    portrayal['nodes'] = [{'size': 6,
                           'color': node_color(agents[0]),
                           'tooltip': "id: {}<br>state: {}".format(agents[0].unique_id, agents[0].state.name),
                           }
                          for (_, agents) in G.nodes.data('agent')]

    portrayal['edges'] = [{'source': source,
                           'target': target,
                           'color': edge_color(*get_agents(source, target)),
                           'width': edge_width(*get_agents(source, target)),
                           }
                          for (source, target) in G.edges]

    return portrayal

network = NetworkModule(network_portrayal, 500, 500, library='d3')
chart = ChartModule([{'Label': 'Burnt', 'Color': '#FF0000'},
                     {'Label': 'Susceptible', 'Color': '#008000'},
                    ])
                     #{'Label': 'Unburnt', 'Color': '#808080'}])

class MyTextElement(TextElement):
    def render(self, model):
        ratio = model.unburnt_susceptible_ratio()
        ratio_text = '&infin;' if ratio is math.inf else '{0:.2f}'.format(ratio)
        burnt_text = str(number_burnt(model))

        return "Burnt Remaining: {}".format( burnt_text)
    
model_params = {
    'num_nodes': UserSettableParameter('slider', 'Number of agents', 10, 10, 100, 1,
                                       description='Choose how many nodes to include in the model'),
    'avg_node_degree': UserSettableParameter('slider', 'Avg Node Degree', 3, 3, 8, 1,
                                             description='Avg Node Degree'),
    'initial_outbreak_size': UserSettableParameter('slider', 'Initial Outbreak Size', 1, 1, 10, 1,
                                                   description='Initial Fire Outbreak Size'),
    'fire_spread_chance': UserSettableParameter('slider', 'Fire Spread Chance', 0.4, 0.0, 1.0, 0.1,
                                                 description='Probability that susceptible neighbor will be burnt'),
    'fire_check_frequency': UserSettableParameter('slider', 'Fire Check Frequency', 0.4, 0.0, 1.0, 0.1,
                                                   description='Frequency the nodes check whether they are burnt by '
                                                               'a fire'),
    'recovery_chance': UserSettableParameter('slider', 'Recovery Chance', 0, 0.0, 1.0, 0.1,
                                             description='Probability that the fire will be put out'),
    'fire_resistance_chance': UserSettableParameter('slider', 'Fire Resistance Chance', 0, 0.0, 1.0, 0.1,
                                                    description='Probability that the node will not be burnt'),
}

server = ModularServer(NetworkOnFire, [network, MyTextElement(), chart], 'Forest Fire Model', model_params)
server.port = 8521

In [5]:
server.launch()

Interface starting at http://127.0.0.1:8521
Socket opened!
{"type":"get_params"}
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"submit_params","param":"num_nodes","value":11}
{"type":"submit_params","param":"num_nodes","value":12}
{"type":"submit_par

{"type":"get_step","step":73}
{"type":"get_step","step":74}
{"type":"get_step","step":75}
{"type":"get_step","step":76}
{"type":"get_step","step":77}
{"type":"get_step","step":78}
{"type":"get_step","step":79}
{"type":"get_step","step":80}
{"type":"get_step","step":81}
{"type":"get_step","step":82}
{"type":"get_step","step":83}
{"type":"get_step","step":84}
{"type":"get_step","step":85}
{"type":"get_step","step":86}
{"type":"get_step","step":87}
{"type":"get_step","step":88}
{"type":"get_step","step":89}
{"type":"get_step","step":90}
{"type":"get_step","step":91}
{"type":"get_step","step":92}
{"type":"get_step","step":93}
{"type":"get_step","step":94}
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"